cattbl
===========

Python modules for working in pandas with various loss data tables used for CAT modelling


### Year Event Loss Table Format
Below shows the basic format of the Year Event Loss Table as a series. Individual event losses are defined by a unique multi-level index. One of the index levels has to refer to the year. The only other requirement is that the series should have an attribute called n_yrs. 

In [1]:
import pandas as pd
yelt = pd.read_csv("tests/_data/example_pareto_poisson_yelt.csv")
yelt = yelt.set_index(['Year', 'EventID', 'DayOfYear'])['Loss']
yelt.attrs['n_yrs'] = 1e5
display(yelt.to_frame())

Loss
Year  EventID DayOfYear              
1     45902   189         1313.996261
      33305   232         2540.574756
2     30597   54          1468.851587
4     10078   63          1677.475382
5     7014    9           1852.600528
...                               ...
99994 10923   222        10846.642495
99995 12281   126         2262.909615
99997 25978   55          1099.812510
      33186   120         3636.796884
99998 37515   49          2485.962160

[49812 rows x 1 columns]

### Convert to a year loss table

In [2]:
from cattbl import yeareventloss
display(yelt.yel.to_ylt().to_frame())

,Loss
Year,
1,3854.571017
2,1468.851587
4,1677.475382
5,34999.439551
6,1221.591205
...,...
99992,1690.163154
99994,12014.849030
99995,2262.909615


### Display various EP curves at specified return periods

In [3]:
fmt = "{:,.0f}"
display(
    yelt.yel
    .to_ep_summaries(is_eef=True,
                     return_periods=(1000, 500, 200, 100, 50, 20, 10))
    .style.format(fmt)
)

display(yelt.yel.to_aal_series().to_frame().style.format(fmt))

CurveType,YearLoss,MaxEventLoss,EventLoss
ReturnPeriod,,,
1000,"494,221","494,221","494,221"
500,"276,674","272,297","272,297"
200,"104,018","101,450","101,450"
100,"51,611","49,340","49,534"
50,"25,875","24,074","24,300"
20,"10,961","9,648","9,884"
10,"5,539","4,708","4,947"


,
AAL,"5,564"
STD,"158,881"


## A differently formatted YELT
In this case there is a separate index level for the Model ID, which categorises groups of events. There are also two loss perspectives stored in separate columns of a dataframe. 

In [4]:
yelts = pd.read_csv("tests/_data/example_two_models_grossnet.csv")
yelts = yelts.set_index(['Year', 'ModelID', 'EventID', 'DayOfYear'])
yelts.attrs['n_yrs'] = 1e5
display(yelts)

GrossLoss  NetLoss
Year   ModelID EventID DayOfYear                      
1      Model1  45902   189        1313.996261      0.0
               33305   232        2540.574756      0.0
2      Model1  30597   54         1468.851587      0.0
4      Model1  10078   63         1677.475382      0.0
5      Model1  7014    9          1852.600528      0.0
...                                       ...      ...
99995  Model2  19599   54         1024.742827      0.0
               40252   362        2469.358250      0.0
99999  Model2  72370   2          3170.787193      0.0
               83257   282        3533.693832      0.0
100000 Model2  48460   315        1016.256966      0.0

[149727 rows x 2 columns]

### EP curves on different partitions of the YELT

In [5]:
return_periods = (10000, 1000, 500, 200, 100, 50, 20, 10)

# AEP on the net loss
yelts['NetLoss'].yel.to_ep_summary(return_periods).to_frame().style.format(fmt)

,Loss
ReturnPeriod,
10000,"120,000"
1000,"120,000"
500,"120,000"
200,"99,561"
100,"42,745"
50,"12,830"
20,0
10,0


In [6]:
# AEP on both loss perspectives
yelts.yel.to_ep_summary(return_periods).style.format(fmt)

,GrossLoss,NetLoss
ReturnPeriod,,
10000,"7,127,715","120,000"
1000,"539,469","120,000"
500,"300,833","120,000"
200,"124,962","99,561"
100,"67,766","42,745"
50,"37,227","12,830"
20,"18,854",0
10,"11,510",0


In [7]:
# OEP and AEP on the net loss
yelts['NetLoss'].yel.to_ep_summaries(return_periods, is_oep=True, is_aep=True).style.format(fmt)

CurveType,YearLoss,MaxEventLoss
ReturnPeriod,,
10000,"120,000","120,000"
1000,"120,000","120,000"
500,"120,000","120,000"
200,"99,561","97,476"
100,"42,745","42,450"
50,"12,830","12,771"
20,0,0
10,0,0


In [8]:
# OEP on the net loss only for each model
yelts['NetLoss'].yel.to_ep_summaries(return_periods, is_aep=False, is_oep=True, is_eef=False, 
                                     splitby='ModelID').style.format(fmt)

In [9]:
# AEP & OEP on both gross and net loss
yelts.yel.to_ep_summaries(return_periods, is_oep=True, is_aep=True).style.format(fmt)

In [10]:
# AEP & OEP on both gross and net loss, split by modelID
(yelts['NetLoss'].yel
 .to_ep_summaries(return_periods[::-1], is_oep=True, is_aep=True, splitby='ModelID')
 .T
).style.format(fmt)

,ReturnPeriod,10,20,50,100,200,500,1000,10000
CurveType,ModelID,,,,,,,,
YearLoss,Model1,0,0,"4,074","29,496","82,403","120,000","120,000","120,000"
MaxEventLoss,Model1,0,0,"4,074","29,340","81,450","120,000","120,000","120,000"
YearLoss,Model2,0,0,0,"1,388","14,418","44,701","82,116","120,000"
MaxEventLoss,Model2,0,0,0,"1,388","14,339","44,701","81,494","120,000"


In [11]:
# AEP & OEP on both gross and net loss, split by modelID
(yelts.yel
 .to_ep_summaries(return_periods[::-1], is_oep=True, is_aep=True, splitby='ModelID')
 .T
 .sort_index(level=['CurveType', 'LossPerspective', 'ModelID', ])
).style.format(fmt)